Drug Ego Networks
Undirected

In [2]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

In [13]:
# read edge list
edgelist = pd.read_pickle('data/subreddit_edges.pkl')

In [34]:
# split networks by subreddit
subs = ['Stims','cannabis','Psychedelics','opiates','benzodiazepines']
for sub in subs:
    # create graph
    G = nx.from_pandas_edgelist(edgelist[edgelist['from'] == sub], source='from', target='subreddit')
    # get largest connected component
    Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
    G0 = G.subgraph(Gcc[0])
    # get ego network
    G_ego = nx.ego_graph(G0, sub, radius=1, center=True, undirected=True, distance=None)
    
    # get nodes and edges
    nodes = list(G_ego.nodes())
    edges = list(G_ego.edges())
    # create dataframe
    df = pd.DataFrame({'from': [i[0] for i in edges], 'to': [i[1] for i in edges]})
    # mean degree
    mean_degree = np.mean(list(dict(G_ego.degree()).values()))
    print(f'{sub} Average degree: ', {mean_degree})
    # average clustering coefficient
    print('Average clustering coefficient: ', nx.average_clustering(G_ego))
    
    # add weights to edges based on number of shared users
    # get list of users who post to each subreddit
    edge_counts = edgelist[edgelist['from'] == sub].groupby(['from','subreddit']).size().reset_index(name = "weight")
    # add weights to edges
    for i, row in edge_counts.iterrows():
        G_ego[row['from']][row['subreddit']]['weight'] = row['weight']
    edge_counts.sort_values(by='weight', ascending=False).head(10)
    # remove nodes with degree less than 2
    # nodes_to_remove = [node for node, degree in G_ego.degree() if degree < 2]
    # G_ego.remove_nodes_from(nodes_to_remove)
    print("info: ", nx.info(G_ego))
    # # save as pickle
    df.to_pickle(f'data/subreddit_ego_{sub}.pkl')
    # # save as csv
    df.to_csv(f'data/subreddit_ego_{sub}.csv', index=False)

Stims Average degree:  {1.9992492492492493}
Average clustering coefficient:  0.0


/var/folders/jj/bt0ykz9n5_16z2gvbybl0tbc0000gn/T/ipykernel_62933/3021883996.py:33: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print("info: ", nx.info(G_ego))


info:  Graph with 2664 nodes and 2663 edges
cannabis Average degree:  {1.999670510708402}
Average clustering coefficient:  0.0
info:  Graph with 6070 nodes and 6069 edges


/var/folders/jj/bt0ykz9n5_16z2gvbybl0tbc0000gn/T/ipykernel_62933/3021883996.py:33: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print("info: ", nx.info(G_ego))


Psychedelics Average degree:  {1.9996276991809383}
Average clustering coefficient:  0.0
info:  Graph with 5372 nodes and 5371 edges


/var/folders/jj/bt0ykz9n5_16z2gvbybl0tbc0000gn/T/ipykernel_62933/3021883996.py:33: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print("info: ", nx.info(G_ego))


opiates Average degree:  {1.9996502885119776}
Average clustering coefficient:  0.0
info:  Graph with 5719 nodes and 5718 edges


/var/folders/jj/bt0ykz9n5_16z2gvbybl0tbc0000gn/T/ipykernel_62933/3021883996.py:33: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print("info: ", nx.info(G_ego))


benzodiazepines Average degree:  {1.9995613073042333}
Average clustering coefficient:  0.0
info:  Graph with 4559 nodes and 4558 edges


/var/folders/jj/bt0ykz9n5_16z2gvbybl0tbc0000gn/T/ipykernel_62933/3021883996.py:33: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print("info: ", nx.info(G_ego))


In [37]:
subs = ['Stims','cannabis','Psychedelics','opiates','benzodiazepines']
for sub in subs:
# read edge list
    df = edgelist[edgelist['from'] == sub]
    # make weights from number of shared users
    df["weight"] = df.groupby(['from','subreddit'])['from'].transform('count')
    # rename columns
    df = df.rename(columns={'from': 'source', 'subreddit': 'target'})
    # export to csv
    df.to_csv(f'data/subreddit_ego_{sub}.csv', index=False)

/var/folders/jj/bt0ykz9n5_16z2gvbybl0tbc0000gn/T/ipykernel_62933/2923910470.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["weight"] = df.groupby(['from','subreddit'])['from'].transform('count')
/var/folders/jj/bt0ykz9n5_16z2gvbybl0tbc0000gn/T/ipykernel_62933/2923910470.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["weight"] = df.groupby(['from','subreddit'])['from'].transform('count')
/var/folders/jj/bt0ykz9n5_16z2gvbybl0tbc0000gn/T/ipykernel_62933/2923910470.py:6: SettingWithCopyWarning: 

---

In [42]:
# get the list of users subreddits
# create edge list by doing a self join to create links between subreddits where a user is active
# agreggate and count the links to get the weight
# remove self loops and reverse links

import json
import pandas as pd
import os
# users df
users = pd.read_pickle('./data/users.pkl')
users = users[['name', 'subreddit', 'type']]

# user subreddits json
path = './data/user_subs/'
dates = ['2023-04-18']

# read each file in dir and append to list
user_subs = pd.DataFrame()
# create new columns
user_subs['name'] = ''
user_subs['subreddit'] = ''
for date in dates:
    files = os.listdir(path + date)
    for file in files:
        with open(path + date + '/' + file) as f:
            user_dict = json.load(f)
            names = user_dict.keys()
            subs = user_dict.values()
            user_subs = pd.concat([user_subs,pd.DataFrame({'name': names, 'subreddit': subs})])
# reset index
user_subs = user_subs.reset_index(drop=True)

n = 4990 users after filtering for potential bots and duplicate 

In [46]:
# create edge list by doing a self join to create links between subreddits where a user is active

# create new columns
user_subs2 = pd.DataFrame()
user_subs2['from'] = ''
user_subs2['to'] = ''
# create edge list
user_subs2['from'] = user_subs['subreddit'].explode()
user_subs2['to'] = user_subs['subreddit'].explode()
# remove self loops
user_subs2 = user_subs2[user_subs2['from'] != user_subs2['to']]
# remove reverse links
# user_subs = user_subs.sort_values(by=['from','to'])
# user_subs = user_subs.drop_duplicates(subset=['from','to'], keep='first')
# # reset index
# user_subs = user_subs.reset_index(drop=True)

# # agreggate and count the links to get the weight
# user_subs = user_subs.groupby(['from','to']).size().reset_index(name = "weight")


In [ ]:
# get the frequency of each pair of subreddit

